In [151]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data\artist_song\artist_song.csv
data\artist_song\artist_song.json
data\billboard\billboard_top_100.csv
data\billboard\billboard_top_100_2020.csv
data\input\songs-2.csv
data\input\top100.json
data\input\top100_old.json
data\lyrics\artist_song_lyrics _bak_2.csv
data\lyrics\artist_song_lyrics.csv
data\lyrics\artist_song_lyrics.xlsx
data\lyrics\artist_song_lyrics_bak_1.csv
data\lyrics\artist_song_lyrics_bak_3.csv
data\lyrics\artist_song_lyrics_bak_4.csv
data\lyrics\artist_song_lyrics_manual.csv
data\lyrics\long_lyrics.csv
data\lyrics\short_lyrics.csv


# Import

In [152]:
top2020 = pd.read_json('data/input/top100.json')
top2020.head()

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
0,1,1,Mariah Carey,All I Want For Christmas Is You,1 Last Week,1 Peak Rank,37 Weeks on Chart,2020-01-04
1,2,2,Brenda Lee,Rockin' Around The Christmas Tree,2 Last Week,2 Peak Rank,32 Weeks on Chart,2020-01-04
2,3,3,Bobby Helms,Jingle Bell Rock,9 Last Week,3 Peak Rank,30 Weeks on Chart,2020-01-04
3,4,4,Burl Ives,A Holly Jolly Christmas,6 Last Week,4 Peak Rank,15 Weeks on Chart,2020-01-04
4,5,5,Post Malone,Circles,3 Last Week,1 Peak Rank,17 Weeks on Chart,2020-01-04


In [153]:
top2019 = pd.read_csv('data/input/songs-2.csv')
top2019.head()

,Unnamed: 0,title,artist,peakPos,lastPos,weeks,isNew,rank,date,year
0,0,Mood,24kGoldn Featuring iann dior,1,1,15,False,1,2020-11-28,2020
1,1,Therefore I Am,Billie Eilish,2,94,2,False,2,2020-11-28,2020
2,2,Positions,Ariana Grande,1,2,4,False,3,2020-11-28,2020
3,3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,False,4,2020-11-28,2020
4,4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,False,5,2020-11-28,2020


# Preparing 2019 Data
### Changing Cols

In [154]:
print(top2020.columns)
print(100*'_')
print(top2019.columns)


Index(['id', 'rank', 'artist', 'song', 'rank_last_week', 'peak_rank',
       'weeks_on_chart', 'date'],
      dtype='object')
____________________________________________________________________________________________________
Index(['Unnamed: 0', 'title', 'artist', 'peakPos', 'lastPos', 'weeks', 'isNew',
       'rank', 'date', 'year'],
      dtype='object')


In [155]:
top2019.rename(columns = {'title':'song', 'peakPos':'peak_rank', 'lastPos':'rank_last_week', 'weeks': 'weeks_on_chart'}, inplace = True)
top2019 = top2019.drop(['Unnamed: 0', 'isNew', 'year'], axis=1)
top2019.head()

,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,Mood,24kGoldn Featuring iann dior,1,1,15,1,2020-11-28
1,Therefore I Am,Billie Eilish,2,94,2,2,2020-11-28
2,Positions,Ariana Grande,1,2,4,3,2020-11-28
3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,4,2020-11-28
4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,5,2020-11-28


### Checking DTypes

In [156]:
top2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song            10000 non-null  object
 1   artist          10000 non-null  object
 2   peak_rank       10000 non-null  int64 
 3   rank_last_week  10000 non-null  int64 
 4   weeks_on_chart  10000 non-null  int64 
 5   rank            10000 non-null  int64 
 6   date            10000 non-null  object
dtypes: int64(4), object(3)
memory usage: 547.0+ KB


### Converting object to datetime

In [157]:
top2019['date'] = pd.to_datetime(top2019['date'], format='%Y-%m-%d')
top2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   song            10000 non-null  object        
 1   artist          10000 non-null  object        
 2   peak_rank       10000 non-null  int64         
 3   rank_last_week  10000 non-null  int64         
 4   weeks_on_chart  10000 non-null  int64         
 5   rank            10000 non-null  int64         
 6   date            10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 547.0+ KB


In [158]:
top2019.head()

,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,Mood,24kGoldn Featuring iann dior,1,1,15,1,2020-11-28
1,Therefore I Am,Billie Eilish,2,94,2,2,2020-11-28
2,Positions,Ariana Grande,1,2,4,3,2020-11-28
3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,4,2020-11-28
4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,5,2020-11-28


### Getting 2019 Charts

In [159]:
top2019 = top2019.loc[(top2019['date'] > '2018-12-31') & (top2019['date'] < '2020-01-04')]
print(top2019.shape)
print(top2019['date'].min())
print(top2019['date'].max())

(5200, 7)
2019-01-05 00:00:00
2019-12-28 00:00:00


# Preparing 2020 Data

### Checking Shape

In [160]:
top2020 = top2020.drop(['id'], axis=1)
print(top2020.shape)
print(top2020['date'].min())
print(top2020['date'].max())

(5300, 7)
2020-01-04 00:00:00
2021-01-02 00:00:00


In [161]:
top2020.head()

,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
0,1,Mariah Carey,All I Want For Christmas Is You,1 Last Week,1 Peak Rank,37 Weeks on Chart,2020-01-04
1,2,Brenda Lee,Rockin' Around The Christmas Tree,2 Last Week,2 Peak Rank,32 Weeks on Chart,2020-01-04
2,3,Bobby Helms,Jingle Bell Rock,9 Last Week,3 Peak Rank,30 Weeks on Chart,2020-01-04
3,4,Burl Ives,A Holly Jolly Christmas,6 Last Week,4 Peak Rank,15 Weeks on Chart,2020-01-04
4,5,Post Malone,Circles,3 Last Week,1 Peak Rank,17 Weeks on Chart,2020-01-04


In [162]:
top2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5300 entries, 0 to 5299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   rank            5300 non-null   int64         
 1   artist          5300 non-null   object        
 2   song            5300 non-null   object        
 3   rank_last_week  5300 non-null   object        
 4   peak_rank       5300 non-null   object        
 5   weeks_on_chart  5300 non-null   object        
 6   date            5300 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 290.0+ KB


In [163]:
top2020['rank_last_week'] = top2020['rank_last_week'].apply(lambda rank_last_week: rank_last_week[:-10]).replace('-', 0)
top2020['peak_rank'] = top2020['peak_rank'].apply(lambda peak_rank: peak_rank[:-10]).replace('-', 0)
top2020['weeks_on_chart'] = top2020['weeks_on_chart'].apply(lambda weeks_on_chart: weeks_on_chart[:-15]).replace('-', 0)

for col in ['rank_last_week', 'peak_rank', 'weeks_on_chart']:
    top2020[col] = pd.to_numeric(top2020[col], errors='coerce')

top2020.sort_values(by=['date', 'rank'], inplace=True)
top2020.reset_index(drop=True, inplace=True)
top2020.insert(loc=0, column='id', value=top2020.index)

# Merge

In [164]:
billboard_top_100 = top2019.merge(top2020, how='outer')
billboard_top_100.shape


(10500, 8)

In [168]:
billboard_top_100.sort_values(by=['date', 'rank'], inplace=True)
billboard_top_100.reset_index(drop=True, inplace=True)
billboard_top_100['id'] = billboard_top_100.index

billboard_top_100 = billboard_top_100[['id', 'song', 'artist', 'peak_rank', 'rank_last_week', 'weeks_on_chart',
       'rank', 'date']]
billboard_top_100.tail()


,id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
10495,10495,Hole In The Bottle,Kelsea Ballerini,72,0,10,96,2021-01-02
10496,10496,"No Body, No Crime",Taylor Swift Featuring HAIM,34,34,2,97,2021-01-02
10497,10497,Moonwalking In Calabasas,DDG,98,0,1,98,2021-01-02
10498,10498,Took Her To The O,King Von,47,0,6,99,2021-01-02
10499,10499,Put Your Records On,Ritt Momney,79,0,10,100,2021-01-02


In [173]:
billboard_top_100.loc[(billboard_top_100['date'] == '2019-08-17')].head()

,id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
3200,3200,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,1,1,23,1,2019-08-17
3201,3201,Bad Guy,Billie Eilish,2,2,19,2,2019-08-17
3202,3202,Senorita,Shawn Mendes & Camila Cabello,2,3,7,3,2019-08-17
3203,3203,Truth Hurts,Lizzo,4,4,14,4,2019-08-17
3204,3204,Talk,Khalid,3,5,26,5,2019-08-17


In [170]:
# Show Top 5 Songs of July 25, 2020 Chart
pd.set_option('display.max_seq_items', None)
lat_table = top2020.loc[(top2020['date'] == "2020-07-25")].head(5)
lat_table

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
2900,2900,1,DaBaby Featuring Roddy Ricch,Rockstar,1,1,13,2020-07-25
2901,2901,2,Juice WRLD x Marshmello,Come & Go,0,2,1,2020-07-25
2902,2902,3,"Jack Harlow Featuring DaBaby, Tory Lanez & Lil...",Whats Poppin,3,2,23,2020-07-25
2903,2903,4,The Weeknd,Blinding Lights,2,1,33,2020-07-25
2904,2904,5,Juice WRLD,Wishing Well,0,5,1,2020-07-25


In [172]:
print(lat_table.to_latex())

\begin{tabular}{lrrllrrrl}
\toprule
{} &    id &  rank &                                             artist &             song &  rank\_last\_week &  peak\_rank &  weeks\_on\_chart &       date \\
\midrule
2900 &  2900 &     1 &                       DaBaby Featuring Roddy Ricch &         Rockstar &               1 &          1 &              13 & 2020-07-25 \\
2901 &  2901 &     2 &                            Juice WRLD x Marshmello &        Come \& Go &               0 &          2 &               1 & 2020-07-25 \\
2902 &  2902 &     3 &  Jack Harlow Featuring DaBaby, Tory Lanez \& Lil... &     Whats Poppin &               3 &          2 &              23 & 2020-07-25 \\
2903 &  2903 &     4 &                                         The Weeknd &  Blinding Lights &               2 &          1 &              33 & 2020-07-25 \\
2904 &  2904 &     5 &                                         Juice WRLD &     Wishing Well &               0 &          5 &               1 & 2020-07-25 \\
\b

In [171]:
billboard_top_100.to_csv('data/billboard/billboard_top_100.csv')
top2020.to_csv('data/billboard/billboard_top_100_2020.csv')